In [1]:
import pandas as pd
import joblib
import numpy as np
import sys
import os
from sklearn.base import BaseEstimator, TransformerMixin

class DateFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, variable):
        self.variable = variable

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for var in self.variable:
            # Aseguramos formato datetime
            X[var] = pd.to_datetime(X[var])
            
            # Extraemos características
            X[var + '_year'] = X[var].dt.year
            X[var + '_month'] = X[var].dt.month
            X[var + '_day'] = X[var].dt.day
            X[var + '_weekday'] = X[var].dt.dayofweek
            
            # Eliminamos la columna original para evitar errores en el modelo
            X.drop(columns=[var], inplace=True)
        return X

print("Librerías importadas y clase DateFeatureExtractor definida.")

Librerías importadas y clase DateFeatureExtractor definida.


In [2]:
# 1. Cargar el Pipeline Entrenado
path_pipeline = '../models/sales_forecasting_pipeline_v1.pkl'

try:
    loaded_pipeline = joblib.load(path_pipeline)
    print(f"✅ Pipeline cargado exitosamente desde: {path_pipeline}")
except Exception as e:
    print(f"❌ Error al cargar el pipeline: {e}")

# 2. Cargar los Datos Originales
df = pd.read_excel('../data/raw/data_sales_forecasting.xlsx', sheet_name='Base de Datos')

# Ordenar por fecha 
df['Fecha'] = pd.to_datetime(df['Fecha'])
df.sort_values('Fecha', inplace=True)
df.reset_index(drop=True, inplace=True)

print(f"Datos cargados: {df.shape[0]} registros totales.")

✅ Pipeline cargado exitosamente desde: ../models/sales_forecasting_pipeline_v1.pkl
Datos cargados: 651372 registros totales.


In [3]:
split_point = int(len(df) * 0.80)

# Creamos el dataset de test (últimos 20%)
df_test = df.iloc[split_point:].copy()

# Separamos las características (X) del objetivo real (y)
X_test = df_test.drop(columns=['Venta_Neta_GTQ'])

y_real = df_test['Venta_Neta_GTQ']

print("--- Generación de Dataset de Inferencia ---")
print(f"Registros para predecir: {len(X_test)}")
print(f"Rango de fechas a predecir: {X_test['Fecha'].min()} hasta {X_test['Fecha'].max()}")

--- Generación de Dataset de Inferencia ---
Registros para predecir: 130275
Rango de fechas a predecir: 2024-04-28 00:00:00 hasta 2024-12-31 00:00:00


In [4]:
print("Generando predicciones...")

predicciones = loaded_pipeline.predict(X_test)

print("¡Predicciones generadas con éxito!")

Generando predicciones...


c:\Users\gabri\anaconda3\envs\env_papd_project\lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


¡Predicciones generadas con éxito!


In [5]:
# Crear un DataFrame con los resultados
resultados_df = X_test.copy() 

# Agregamos la predicción
resultados_df['Venta_Predicha'] = predicciones

# Agregamos la venta real para referencia 
resultados_df['Venta_Real'] = y_real

# Calculamos el error absoluto para cada fila 
resultados_df['Error_Absoluto'] = abs(resultados_df['Venta_Real'] - resultados_df['Venta_Predicha'])

# Seleccionamos columnas clave para el reporte final
columnas_reporte = [
    'Fecha', 
    'No_Tienda', 
    'Descripcion_Cupon', 
    'Venta_Real', 
    'Venta_Predicha', 
    'Error_Absoluto'
]

df_export = resultados_df[columnas_reporte]

# Mostrar las primeras filas
print(df_export.head())

            Fecha  No_Tienda  \
521097 2024-04-28      14012   
521098 2024-04-28      14010   
521099 2024-04-28      14002   
521100 2024-04-28      14000   
521101 2024-04-28      14002   

                                        Descripcion_Cupon  Venta_Real  \
521097  Mediana de especialidad + orilla de queso por Q75       59.53   
521098  Grande de Queso, 1 ING o Esp + Mediana de Ques...       68.79   
521099                 Pizzas Grandes Esp Q80 C/U 2 o Mas      142.86   
521100                        Mediana de Especialidad Q50       44.64   
521101                                         2 o Mas CC       71.43   

        Venta_Predicha  Error_Absoluto  
521097       59.530000    1.705303e-13  
521098       69.604389    8.143889e-01  
521099      142.860000    3.694822e-13  
521100       44.759100    1.191000e-01  
521101       71.876000    4.460000e-01  


In [6]:
# Definir ruta de salida
output_csv_path = '../data/processed/predicciones_test_set.csv'

# Asegurarse que la carpeta exista
os.makedirs('../data/processed', exist_ok=True)

df_export.to_csv(output_csv_path, index=False)

print(f"✅ Archivo de predicciones generado exitosamente en: {output_csv_path}")

✅ Archivo de predicciones generado exitosamente en: ../data/processed/predicciones_test_set.csv
